In [1]:
import codecs
import glob
import math
import os
from decimal import Decimal
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
file_name = "data/input/XXX"
df = pd.read_table(file_name)

In [3]:
initial_key = "Summary of Optimized"
end_key = "---------------------------------------------------"
Eigenvalues = "Eigenvalues"
torsion1 = "d15"
torsion2 = "d13"
# other_value = "xxx"

add_value = float(-632.0)

In [4]:
# エネルギーを取る場所と角度を取る場所が違うことに注意
E_initial_key = "HF="
E_end_key = "RMSD="

E_initial_key_df = df[df[df.columns[0]].str.contains(E_initial_key, case=False)]
E_initial_key_index = E_initial_key_df.index[0]
df_E = df.iloc[E_initial_key_index:]
# initial_keyを含む行以降のend_keyを含む行を抜き出す
E_end_key_df = df[df[df.columns[0]].str.contains(E_end_key)]
# dfを更新しているのでここでの行番号は0から始まる
E_end_key_index = E_end_key_df.index[0] - E_initial_key_index
df_E = df_E.iloc[:E_end_key_index + 1]

df_E = df_E.rename(columns={df_E.columns[0]: 'Col_1'})
df_E_list = ｄｆ_E['Col_1'].to_list()
df_E_str = ','.join(df_E_list)
df_E_str = df_E_str.split(E_initial_key)[-1]
df_E_str = df_E_str.split(E_end_key)[0].split(',')
df_E_str = ''.join(df_E_str).split(' ')
df_E_str = ''.join(df_E_str).split('|')[0]
df_E_str_list = df_E_str.split("-")[1:]

Eigenvalues_list = []
for n in range(len(df_E_str_list)):
    list_n = df_E_str_list[n]
    list_n = float(list_n) * -1
    Eigenvalues_list.append(list_n)
    

# initial_keyを含む行を抜き出す
initial_key_df = df[df[df.columns[0]].str.contains(initial_key, case=False)]
initial_key_index = initial_key_df.index[0]
df = df.iloc[initial_key_index:]
# initial_keyを含む行以降のend_keyを含む行を抜き出す
end_key_df = df[df[df.columns[0]].str.contains(end_key)]
# dfを更新しているのでここでの行番号は0から始まる
end_key_index = end_key_df.index[0] - initial_key_index
df = df.iloc[:end_key_index]

# カラム名をわかりやすく変更
df = df.rename(columns={df.columns[0]: 'Col_1'})



# d15を含む行を全て抜き出す
# torsion1_listへ格納

df_t1 = df[df[df.columns[0]].str.contains(torsion1, case=False)]
torsion1_list_str = []
for t1_value in df_t1.iloc[0][0].split(' '):
    if t1_value != '':
        torsion1_list_str.append(t1_value)

torsion1_list = []

for n in range(len(df_t1)):
    list_n = df_t1.iloc[n][0].split(' ')
    list_t1 = []
    for t1_value in list_n:
        if t1_value != '':
            list_t1.append(t1_value)
    list_t1 = list_t1[1:]
    for m in range(len(list_t1)):
        torsion1_list.append(list_t1[m])

# 数値化
torsion1_list = [float(s) for s in torsion1_list]

# d13を含む行を全て抜き出す
# torsion2_listへ格納
df_t2 = df[df[df.columns[0]].str.contains(torsion2, case=False)]
torsion2_list_str = []
for t2_value in df_t2.iloc[0][0].split(' '):
    if t2_value != '':
        torsion2_list_str.append(t2_value)

torsion2_list = []

for n in range(len(df_t2)):
    list_n = df_t2.iloc[n][0].split(' ')
    list_t2 = []
    for t2_value in list_n:
        if t2_value != '':
            list_t2.append(t2_value)
    list_t2 = list_t2[1:]
    for m in range(len(list_t2)):
        torsion2_list.append(list_t2[m])

# 数値化
torsion2_list = [float(s) for s in torsion2_list]

In [5]:
# 一つのdfにまとめる
all_list = list(zip(Eigenvalues_list, torsion1_list, torsion2_list))
df_all = pd.DataFrame(data=all_list, columns=[Eigenvalues, torsion1, torsion2])
# 表形式のdfにする
df_output = pd.DataFrame()
torsion1_unique_list = sorted(list(set(df_all[torsion1])))
for n in torsion1_unique_list:
    df_t1_n = df_all[df_all[torsion1]==n]
    df_t1_n = pd.DataFrame(data=list(df_t1_n[Eigenvalues]), index=list(df_t1_n[torsion2]), columns=[n]).T
    df_output = pd.concat([df_output, df_t1_n])

In [6]:
# 出力確認
df_output

,0.0,15.0,30.0,45.0,60.0,75.0,90.0,105.0,120.0,135.0,...,210.0,225.0,240.0,255.0,270.0,285.0,300.0,315.0,330.0,345.0
0.0,-632.137437,-632.137461,-632.137152,-632.136062,-632.134275,-632.132435,-632.131554,-632.132313,-632.134131,-632.135950,...,-632.137106,-632.135950,-632.134131,-632.132313,-632.131554,-632.132436,-632.134275,-632.136062,-632.137152,-632.137461
15.0,-632.135494,-632.135459,-632.134970,-632.133670,-632.131779,-632.130123,-632.129748,-632.130908,-632.132757,-632.134406,...,-632.134938,-632.133596,-632.131679,-632.130040,-632.129801,-632.131070,-632.132936,-632.134539,-632.135384,-632.135539
30.0,-632.129448,-632.129358,-632.128705,-632.127239,-632.125348,-632.124031,-632.124169,-632.125530,-632.127278,-632.128689,...,-632.128702,-632.127213,-632.125288,-632.123987,-632.124261,-632.125715,-632.127472,-632.128823,-632.129439,-632.129484
45.0,-632.119046,-632.118931,-632.118158,-632.116590,-632.114811,-632.113926,-632.114423,-632.115783,-632.117294,-632.118436,...,-632.118194,-632.116616,-632.114781,-632.113918,-632.114533,-632.115957,-632.117476,-632.118561,-632.118986,-632.119008
60.0,-632.104734,-632.104697,-632.103906,-632.102329,-632.100706,-632.100129,-632.100671,-632.101774,-632.102972,-632.103880,...,-632.103978,-632.102398,-632.100694,-632.100137,-632.100752,-632.101905,-632.103127,-632.103991,-632.104351,-632.104504
75.0,-632.088640,-632.088856,-632.088250,-632.086816,-632.085340,-632.084731,-632.084860,-632.085486,-632.086357,-632.087124,...,-632.088332,-632.086905,-632.085333,-632.084722,-632.084888,-632.085559,-632.086474,-632.087232,-632.087675,-632.088096
90.0,-632.073375,-632.073890,-632.073646,-632.072684,-632.071641,-632.070689,-632.070113,-632.070185,-632.070779,-632.071466,...,-632.073698,-632.072730,-632.071623,-632.070658,-632.070101,-632.070213,-632.070862,-632.071559,-632.072059,-632.072611
105.0,-632.086590,-632.085782,-632.085094,-632.084707,-632.083810,-632.082445,-632.081503,-632.081755,-632.083275,-632.085205,...,-632.085019,-632.084614,-632.083661,-632.082352,-632.081489,-632.081748,-632.083187,-632.085081,-632.086511,-632.086980
120.0,-632.098771,-632.097993,-632.097233,-632.096491,-632.095251,-632.093748,-632.092800,-632.093237,-632.095042,-632.097162,...,-632.097127,-632.096344,-632.095084,-632.093651,-632.092788,-632.093236,-632.094965,-632.097061,-632.098589,-632.099117
135.0,-632.108760,-632.108058,-632.107335,-632.106414,-632.104978,-632.103364,-632.102389,-632.102965,-632.104949,-632.107111,...,-632.107228,-632.106252,-632.104812,-632.103270,-632.102378,-632.102969,-632.104902,-632.107055,-632.108564,-632.109075


In [7]:
# 問題なければ出力
# 文字化けするなら encoding='utf_8_sig' をつける
result_path = "data/output/result.csv"
df_output.to_csv("data/output/result.csv")